## Text Classification

In [96]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
from hazm import *
import itertools
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
import glob, os
from pandas_confusion import ConfusionMatrix
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score,recall_score,precision_score,classification_report,confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import numpy as np
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
import pickle
import joblib
import pandas as pd

In [97]:

stwdfa = ['همچنان', 'مدت', 'چیز', 'سایر', 'جا', 'طی', 'کل', 'کنونی', 'بیرون','های', 'مثلا', 'کامل','ها', 'کاملا','گیرد','شود','است', 'آنکه', 
            'موارد', 'واقعی', 'امور', 'اکنون', 'بطور', 'بخشی', 'تحت', 'چگونه', 'عدم', 'نوعی', 'حاضر', 'وضع', 'مقابل', 'کنار', 'خویش', 'نگاه', 'درون',
            'زمانی', 'بنابراین', 'تو', 'خیلی', 'بزرگ', 'خودش', 'جز', 'اینجا', 'مختلف', 'توسط', 'نوع', 'همچنین', 'آنجا', 'قبل', 'جناح', 'اینها', 'طور', 'شاید',
            'ایشان', 'جهت', 'طریق', 'مانند', 'پیدا', 'ممکن', 'کسانی', 'جای', 'کسی', 'غیر', 'بی', 'قابل', 'درباره', 'جدید', 'وقتی', 'اخیر', 'چرا', 'بیش',
            'روی', 'طرف', 'جریان', 'زیر', 'آنچه', 'البته', 'فقط', 'چیزی', 'چون', 'برابر', 'هنوز', 'بخش', 'زمینه', 'بین', 'بدون', 'استفاد', 'همان', 'نشان',
            'بسیاری', 'بعد', 'عمل', 'روز', 'اعلام', 'چند', 'آنان', 'بلکه', 'امروز', 'تمام', 'بیشتر', 'آیا', 'برخی', 'علیه', 'دیگری', 'ویژه', 'گذشته', 'انجام',
            'حتی', 'داده', 'راه', 'سوی', 'ولی', 'زمان', 'حال', 'تنها', 'بسیار', 'یعنی', 'عنوان', 'همین', 'هبچ', 'پیش', 'وی', 'یکی', 'اینکه', 'وجود'
            , 'شما', 'پس', 'چنین', 'میان', 'مورد', 'چه', 'اگر', 'همه', 'نه', 'دیگر', 'آنها', 'باید', 'هر', 'او', 'ما', 'من', 'تا', 'نیز', 'اما', 
            'یک', 'خود', 'بر', 'یا', 'هم','ای', 'را','دارد', 'این',"می", 'با','دارد',',',',','.', 'آن', 'برای', 'و', 'در', 'به', 'که', 'از']


In [98]:
stwden=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself',
'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them',
'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was',
'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or',
'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before',
'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once',
'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no',
'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should',
"should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't",
'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't",
'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won',
"won't", 'wouldn', "wouldn't"]

In [6]:
def clean(text):
    normalizer = Normalizer()
    # Remove URLs, User Mentions and Hashtags and Retweet and Number
    text = normalizer.normalize(text)
    text = ''.join(ch for ch, _ in itertools.groupby(text))
    # Remove Punctuation
    #text = re.sub(r'[^\w\s]', '', text)
    return text

In [86]:
emojien={'angry':['>:-[', ':-@', ']:[', '=@', '>=-[', ']:-(', ']=-[', ']=(', ' ]:-[', ']:(', ':@', '>:(', '>=-(', ']=[', '>=(', '=-@', '>:-(', ']=-(', '>=[', '>:['],
   'happy':[':")', 'X-)', 'X)', ':,-)', 'x-]', "='D", '=,-]', '=)', ':,)', '="D', '=,D', '=-")', '=D', "='-D", ':"D', ':,D', ":'-)", "=')", ':D', '=-"D', ':-)', 'x]', '=-D', '=,)', ':)', 'X-D', ':-]', ':-")', '=,-D', 'x)', '="]', ':,]', ':-D', '=")', ':,-D', '=-]', 'X]', ":']", ":'-]", '=,]', 'X-]', ':-"]', ':,-]', "='-)", '=-)', ':"]', ":')", '=]', ':]', '=,-)', "=']", 'x-D', ':-"D', '=-"]', "='-]", 'x-)', 'xD','Xp', "'=-]", ',:D', ';-p', "':-)", ';-d', '=p', ',=-)', "'=]", "':-D", '=-P', 'Xd', 'X-d', ';]', ';-D', 'x-p', "'=-D", ',=]', "':-]", ':p', 'x-P', ',=-]', "':]", ',:-)', "':D", 'X-p', ':P', "'=-)", 'xP', "':)", '=-d', 'X-P', ':-d', ';P', ';-P', ',:]', ';p', ';)', ',=)', ',:-]', "'=D", ';-)', ';D', '=d', ',:-D', "'=)", '=P', ',:)', ',=-D', ';-]', ':-p', ':d', ';d', 'x-d', ',=D', '=-p', ':-P'],
   'confused':[':/', ':-/', '=/', '=-/'],
   'sad':['</3',':,(',':,[',':,|',':,-(',':,-[',':,-|',":'(",":'[",":'|",":'-(",":'-[",":'-|","=,(", "=,["," =,|", "=,-(", "=,-[", "=,-|", "='(", "='[", "='|", "='-(", "='-[", "='-|"],
   'disappointed':[':(', ':[', ':-(', ':-[',' =(', '=[',' =-(', '=-['],
   'innocent':['0=]', 'o:]', 'O=D', 'O=-)', '0:-D', '0:-]', 'o=-]', 'O=)', '0=)', '0:)', '0:D', 'O:-]', 'O:D', 'o:-)', 'o:D', 'o:-]', '0=D', 'O=-]', 'o=-D', 'o=-)', 'O:-)', 'o:)', 'o=D', 'O=-D', 'O:]', 'o=]', ' O:-D', 'o:-D', '0:-)', '0=-]', '0:]', 'O:)', 'o=)', '0=-D', '0=-)', 'O=]'],
   'love':[':*', ':-*', '=*', '=-*','<3'],
   'man':[':3', ':-3', '=3', '=-3',';3', ';-3', 'x3', 'x-3', 'X3', 'X-3'],
   'neutral_face':[':|', ':-|', '=|', '=-|'],
   'mute':[':-'],
   'surprise':[':o', ':O', ':0', ':-o', ':-O', ':-0', '=o', '=O', '=0', '=-o', '=-O', '=-0'],
   'naughty':[']:]', ']:-]', ']=-)', ']=D', ']:-)', ']:)', ']=)', ']=]', ']=-]', ']=-D', ']:-D', ']:D'],
   'very sad':[":',(", "=,'(", ":',-(", "=,'[", "=',-[", ":,'[", "=,'-(", "=,'-[", "=',-(", ":,'(", ":,'-(", ":',[", "=',[", ":',-[", "=',(", ":,'-["],
   'cool':['8)', '8]', '8D',' 8-)', '8-]', '8-D', 'B)', 'B]', 'B-)', 'B-]', 'B-D'],
   'embarrassing':[',:(', ',:[', ',:-(', ',:-[',' ,=(', ',=[', ',=-(', ',=-[', "':(", "':[", "':-(", "':-[", "'=(", "'=[", "'=-(", "'=-["],
   'unamused':[':$', ':s', ':z', ':S', ':Z', ':-$', ':-s', ':-z', ':-S', ':-Z', '=$', '=s', '=z', '=S', '=Z', '=-$', '=-s', '=-z', '=-S', '=-Z'],  
    }

In [87]:
emojifa={'خشمگین':['>:-[', ':-@', ']:[', '=@', '>=-[', ']:-(', ']=-[', ']=(', ' ]:-[', ']:(', ':@', '>:(', '>=-(', ']=[', '>=(', '=-@', '>:-(', ']=-(', '>=[', '>:['],
   'خوشحال':[':")', 'X-)', 'X)', ':,-)', 'x-]', "='D", '=,-]', '=)', ':,)', '="D', '=,D', '=-")', '=D', "='-D", ':"D', ':,D', ":'-)", "=')", ':D', '=-"D', ':-)', 'x]', '=-D', '=,)', ':)', 'X-D', ':-]', ':-")', '=,-D', 'x)', '="]', ':,]', ':-D', '=")', ':,-D', '=-]', 'X]', ":']", ":'-]", '=,]', 'X-]', ':-"]', ':,-]', "='-)", '=-)', ':"]', ":')", '=]', ':]', '=,-)', "=']", 'x-D', ':-"D', '=-"]', "='-]", 'x-)', 'xD','Xp', "'=-]", ',:D', ';-p', "':-)", ';-d', '=p', ',=-)', "'=]", "':-D", '=-P', 'Xd', 'X-d', ';]', ';-D', 'x-p', "'=-D", ',=]', "':-]", ':p', 'x-P', ',=-]', "':]", ',:-)', "':D", 'X-p', ':P', "'=-)", 'xP', "':)", '=-d', 'X-P', ':-d', ';P', ';-P', ',:]', ';p', ';)', ',=)', ',:-]', "'=D", ';-)', ';D', '=d', ',:-D', "'=)", '=P', ',:)', ',=-D', ';-]', ':-p', ':d', ';d', 'x-d', ',=D', '=-p', ':-P'],
   'سردرگم':[':/', ':-/', '=/', '=-/'],
   'ناراحت':['</3',':,(',':,[',':,|',':,-(',':,-[',':,-|',":'(",":'[",":'|",":'-(",":'-[",":'-|","=,(", "=,["," =,|", "=,-(", "=,-[", "=,-|", "='(", "='[", "='|", "='-(", "='-[", "='-|"],
   'ناامید':[':(', ':[', ':-(', ':-[',' =(', '=[',' =-(', '=-['],
   'معصومانه':['0=]', 'o:]', 'O=D', 'O=-)', '0:-D', '0:-]', 'o=-]', 'O=)', '0=)', '0:)', '0:D', 'O:-]', 'O:D', 'o:-)', 'o:D', 'o:-]', '0=D', 'O=-]', 'o=-D', 'o=-)', 'O:-)', 'o:)', 'o=D', 'O=-D', 'O:]', 'o=]', ' O:-D', 'o:-D', '0:-)', '0=-]', '0:]', 'O:)', 'o=)', '0=-D', '0=-)', 'O=]'],
   'عشق':[':*', ':-*', '=*', '=-*','<3'],
   'مرد':[':3', ':-3', '=3', '=-3',';3', ';-3', 'x3', 'x-3', 'X3', 'X-3'],
   'چهره خنثی':[':|', ':-|', '=|', '=-|'],
   'ساکت':[':-'],
   'متعجب':[':o', ':O', ':0', ':-o', ':-O', ':-0', '=o', '=O', '=0', '=-o', '=-O', '=-0'],
   'شیطون':[']:]', ']:-]', ']=-)', ']=D', ']:-)', ']:)', ']=)', ']=]', ']=-]', ']=-D', ']:-D', ']:D'],
   'بسیار ناراحت':[":',(", "=,'(", ":',-(", "=,'[", "=',-[", ":,'[", "=,'-(", "=,'-[", "=',-(", ":,'(", ":,'-(", ":',[", "=',[", ":',-[", "=',(", ":,'-["],
   'خونسرد':['8)', '8]', '8D',' 8-)', '8-]', '8-D', 'B)', 'B]', 'B-)', 'B-]', 'B-D'],
   'شرم آور':[',:(', ',:[', ',:-(', ',:-[',' ,=(', ',=[', ',=-(', ',=-[', "':(", "':[", "':-(", "':-[", "'=(", "'=[", "'=-(", "'=-["],
   'خسته کننده':[':$', ':s', ':z', ':S', ':Z', ':-$', ':-s', ':-z', ':-S', ':-Z', '=$', '=s', '=z', '=S', '=Z', '=-$', '=-s', '=-z', '=-S', '=-Z'],  
   
   }

In [88]:
text="hi ali where are you :)))))"

In [89]:

from langdetect import detect,detect_langs  


In [90]:
text2='  سلام علی  :)))) گفتم بیا بریم بیرون . فردا میای بیرون :d'

In [99]:
def emoji_detect(text):
    text_list=text.split()
    clean_list=[]
    if detect(text)=='en':

        for item in text_list:
            if item not in stwden:
                clean_list.append(''.join(ch for ch, _ in itertools.groupby(item)))
            for item in emojien:
                for i in emojien[item]:
                    for c in clean_list:
                        if i==c:
                            clean_list = list(map(lambda x: x.replace(c, '('+item+")"), clean_list))
    elif detect(text)=='fa':
        for item in text_list:
            if item not in stwdfa:
                clean_list.append(''.join(ch for ch, _ in itertools.groupby(item)))
        for item in emojifa:
            for i in emojifa[item]:
                for c in clean_list:
                    if i==c:
                        clean_list = list(map(lambda x: x.replace(c, '('+item+")"), clean_list))
    return ' '.join(clean_list)
        

In [101]:
emoji_detect(text)

'hi ali (happy)'